# Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

<a id='content'></a>
## Оглавление

### Открытие и предобработка данных 
* [Загрузка датасета и проверка его на наличие дубликатов, пропусков и соответствие типам данных, а также правильность выгрузки](#start)
* [Лемматизация](#part_0.1)
* [Категоризация](#part_0.2)

### Анализ данных
* [Есть ли зависимость между наличием детей и возвратом кредита в срок?](#part_1.1)
* [Есть ли зависимость между семейным положением и возвратом кредита в срок?](#part_1.2)
* [Есть ли зависимость между уровнем дохода и возвратом кредита в срок?](#part_1.3)
* [Как разные цели кредита влияют на его возврат в срок?](#part_1.4)

### Вывод
* [Сделаем общий вывод](#finish)

# Открытие и предобработка данных 
<a id='start'></a>
Загрузка датасета и проверка его на наличие дубликатов, пропусков и соответствие типам данных, а также правильность выгрузки

[*Назад к оглавлению*](#content)

In [1]:
import pandas as pd
from pymystem3 import Mystem
df = pd.read_csv('/datasets/data.csv')
df.head(20)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


Общая информация о данных таблицы df.

In [2]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


Рассмотрим полученную информацию подробнее.

Всего в таблице 12 столбцов, типы данных столбцов - int, float, object.

Подробно разберём, какие в *df* столбцы и какую информацию они содержат:

* children — количество детей в семье
* days_employed — общий трудовой стаж в днях
* dob_years — возраст клиента в годах
* education — уровень образования клиента
* education_id — идентификатор уровня образования
* family_status — семейное положение
* family_status_id — идентификатор семейного положения
* gender — пол клиента
* income_type — тип занятости
* debt — имел ли задолженность по возврату кредитов
* total_income — ежемесячный доход
* purpose — цель получения кредита

Количество значений в столбцах различается. Это говорит о том, что в данных есть null значения.
Также сразу бросается в глаза, что некоторая информация предоставлена в разных регистрах, что может затруднить анализ (столбец *education*). Также видны ошибки в данных в столбце *days_employed*.

Проверим число пропущенных значений в таблице.

In [3]:
df.isnull().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

Мы видим отсутствующие значения в столбцах *days_employed* и *total_income*.
Это может произойти из-за ошибки в выгрузке или из-за отстутвия данных в базе. Необходимо сообщить об этих проблемах коллегам, отвтетсвенным за данную информацию.

Значения стажа работы для задач нашего проекта не особо существенны, поэтому пропущенные значения можно принять как среднее арифметическое остальных значений.

In [4]:
df['days_employed'] = df['days_employed'].fillna(df[df['days_employed'] != None]['days_employed'].mean())
df.isnull().sum()

children               0
days_employed          0
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

Пропущенных значений в столбце *days_employed* не осталось. Теперь необходимо разобраться со столбцом *total_income*.

Для одной из задач нашего проекта, а именно "*Есть ли зависимость между уровнем дохода и возвратом кредита в срок?*", нам необходимы значения ежемесячного дохода клиентов банка.
Так как число пропущенных значений достаточно велико (10%), я считаю некорректным использовать данные данных клиентов для решения конкретно этой задачи. Для прочих же задач, где уровень дохода не важен, их можно использовать.

Поэтому предлагаю следующие действия:
* Создать новый столбец (*total_income_id*) с маркером, использовать ли данные клиента для расчета зависимости между уровнем дохода и возвратом кредита в срок
* Все пропущенные данные в столбце *total_income* заменить на медианное значение данного столбца
* Все задачи проекта, кроме зависимости между уровнем дохода и возвратом кредита в срок считать на всех данных DataFrame
* Задачу "*Есть ли зависимость между уровнем дохода и возвратом кредита в срок?*" посчитать на тех клиентах, где не было пропуска данных о доходе

Создадим функцию, которая будет заполнять новый столбец *total_income_id* с маркером и применим *df*.

In [5]:
def new_column_isnull(row):
    if pd.isna(row['total_income']) == True:
        return(0)
    else:
        return(1)

df['total_income_id'] = df.apply(new_column_isnull, axis=1)
df.head(20)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,total_income_id
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья,1
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля,1
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья,1
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование,1
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу,1
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья,1
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем,1
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование,1
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы,1
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи,1


Теперь можем заменить пустые значения в столбце *total_income* на медианное значение.

In [6]:
df['total_income'] = df['total_income'].fillna(df[df['total_income'] != None]['total_income'].median())
df.isnull().sum()

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
total_income_id     0
dtype: int64

Еще раз посмотрим на формат данных в столбцах таблицы.

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 13 columns):
children            21525 non-null int64
days_employed       21525 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null float64
purpose             21525 non-null object
total_income_id     21525 non-null int64
dtypes: float64(2), int64(6), object(5)
memory usage: 2.1+ MB


Заменим типы данных *float* на *int*. Это позволит сэкономить нам память, в то время как на результаты повлияет незначительно. К тому же данные типа *int* будет проще воспринимать, глядя на таблицу.
Будем использовать метод **astype()**.

In [8]:
df['days_employed'] = df['days_employed'].astype('int')
df['total_income'] = df['total_income'].astype('int')
df.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,total_income_id
0,1,-8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,1
1,1,-4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,1
2,0,-5623,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,1
3,3,-4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,1
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,1
5,0,-926,27,высшее,0,гражданский брак,1,M,компаньон,0,255763,покупка жилья,1
6,0,-2879,43,высшее,0,женат / замужем,0,F,компаньон,0,240525,операции с жильем,1
7,0,-152,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823,образование,1
8,2,-6929,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856,на проведение свадьбы,1
9,0,-2188,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425,покупка жилья для семьи,1


In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 13 columns):
children            21525 non-null int64
days_employed       21525 non-null int64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null int64
purpose             21525 non-null object
total_income_id     21525 non-null int64
dtypes: int64(8), object(5)
memory usage: 2.1+ MB


Теперь нам необходимо проверить данные на дубликаты.

Начнем с ручного поиска дубликатов с учётом регистра. Для начала проверим уникальные значения в столбцах *education, family_status, gender, income_type, purpose*.

In [10]:
df['education'].unique()

array(['высшее', 'среднее', 'Среднее', 'СРЕДНЕЕ', 'ВЫСШЕЕ',
       'неоконченное высшее', 'начальное', 'Высшее',
       'НЕОКОНЧЕННОЕ ВЫСШЕЕ', 'Неоконченное высшее', 'НАЧАЛЬНОЕ',
       'Начальное', 'Ученая степень', 'УЧЕНАЯ СТЕПЕНЬ', 'ученая степень'],
      dtype=object)

In [11]:
df['family_status'].unique()

array(['женат / замужем', 'гражданский брак', 'вдовец / вдова',
       'в разводе', 'Не женат / не замужем'], dtype=object)

In [12]:
df['gender'].unique()

array(['F', 'M', 'XNA'], dtype=object)

In [13]:
df['income_type'].unique()

array(['сотрудник', 'пенсионер', 'компаньон', 'госслужащий',
       'безработный', 'предприниматель', 'студент', 'в декрете'],
      dtype=object)

In [14]:
df['purpose'].unique()

array(['покупка жилья', 'приобретение автомобиля',
       'дополнительное образование', 'сыграть свадьбу',
       'операции с жильем', 'образование', 'на проведение свадьбы',
       'покупка жилья для семьи', 'покупка недвижимости',
       'покупка коммерческой недвижимости', 'покупка жилой недвижимости',
       'строительство собственной недвижимости', 'недвижимость',
       'строительство недвижимости', 'на покупку подержанного автомобиля',
       'на покупку своего автомобиля',
       'операции с коммерческой недвижимостью',
       'строительство жилой недвижимости', 'жилье',
       'операции со своей недвижимостью', 'автомобили',
       'заняться образованием', 'сделка с подержанным автомобилем',
       'получение образования', 'автомобиль', 'свадьба',
       'получение дополнительного образования', 'покупка своего жилья',
       'операции с недвижимостью', 'получение высшего образования',
       'свой автомобиль', 'сделка с автомобилем',
       'профильное образование', 'высшее об

Можем заметить, что в столбце *education* некоторые значения дублируются в другом регистре. Чтобы решить эту проблему используем метод **str.lower()**.

In [15]:
df['education'] = df['education'].str.lower()
df['education'].unique()

array(['высшее', 'среднее', 'неоконченное высшее', 'начальное',
       'ученая степень'], dtype=object)

Теперь мы мождем посмотреть число полных дубликатов строк.

In [16]:
df.duplicated().sum()

71

Используем метод **drop_duplicates()** для удаления дубликатов. Также используем **reset_index(drop=True)** для того чтобы не создавать дополнительный столбец со старыми значениями индексов, а переназначить их по порядку.

In [17]:
df = df.drop_duplicates().reset_index(drop=True)
df.duplicated().sum()

0

Так как у нас нет уникальных идентификаторов клиентов, мы не можем побробнее рассмотреть все возможные дубликаты.
Поэтому мы смогли удалить лишь те строки, которые полностью дублировались.

Появление дубликатов могло быть связано как с "человеческим фактором", таким как повторное занесение клиента в базу, так и с проблемами выгрузки. На это стоит обратить внимание.

### Лемматизация
<a id='part_0.1'></a>

Теперь нам необходимо разбить цели получения кредита на категории. Для этого мы применим лемматизацию.

In [18]:
purpose_unique = df['purpose'].unique()
purpose_unique

array(['покупка жилья', 'приобретение автомобиля',
       'дополнительное образование', 'сыграть свадьбу',
       'операции с жильем', 'образование', 'на проведение свадьбы',
       'покупка жилья для семьи', 'покупка недвижимости',
       'покупка коммерческой недвижимости', 'покупка жилой недвижимости',
       'строительство собственной недвижимости', 'недвижимость',
       'строительство недвижимости', 'на покупку подержанного автомобиля',
       'на покупку своего автомобиля',
       'операции с коммерческой недвижимостью',
       'строительство жилой недвижимости', 'жилье',
       'операции со своей недвижимостью', 'автомобили',
       'заняться образованием', 'сделка с подержанным автомобилем',
       'получение образования', 'автомобиль', 'свадьба',
       'получение дополнительного образования', 'покупка своего жилья',
       'операции с недвижимостью', 'получение высшего образования',
       'свой автомобиль', 'сделка с автомобилем',
       'профильное образование', 'высшее об

Выделим следующие категории:
* жилье/недвижимость
* автомобиль
* образование
* свадьба

Разделим наши уникальные цели получения кредита на данные категории и составим словарь.

Создадим список *purpose_category* с категориями кредитов. Для каждого из уникальных значений целей кредита мы применим лемматизацию функцией **m.lemmatize()**. Далее в зависимости от наличия в лемматизированом списке ключевых слов мы внесем в *purpose_category* названия категорий.

In [19]:
m = Mystem()

purpose_category = []

for item in purpose_unique:
    lemmas = m.lemmatize(item)
    if 'жилье' in lemmas:
        purpose_category.append('жилье/недвижимость')
    if 'недвижимость' in lemmas:
        purpose_category.append('жилье/недвижимость')
    if 'автомобиль' in lemmas:
        purpose_category.append('автомобиль')
    if 'образование' in lemmas:
        purpose_category.append('образование')
    if 'свадьба' in lemmas:
        purpose_category.append('свадьба')

Проверим, что длина *purpose_category* соответствует длине *purpose_unique*.

In [20]:
len(purpose_category) == len(purpose_unique)

True

Создадим DataFrame со словарем, где укажем первоначальные уникальные значения целей кредитов и соответствующие им категории.

In [21]:
purpose_dict = pd.DataFrame(data = purpose_unique, columns = ['p_unique'])
purpose_dict['p_category'] = purpose_category
purpose_dict

,p_unique,p_category
0,покупка жилья,жилье/недвижимость
1,приобретение автомобиля,автомобиль
2,дополнительное образование,образование
3,сыграть свадьбу,свадьба
4,операции с жильем,жилье/недвижимость
5,образование,образование
6,на проведение свадьбы,свадьба
7,покупка жилья для семьи,жилье/недвижимость
8,покупка недвижимости,жилье/недвижимость
9,покупка коммерческой недвижимости,жилье/недвижимость


**Вывод**

Мы смогли создать словарь, котоый поможет нам в категоризации целей кредитов в дальнейшем. Несмотря на то, что первоначальных целей кредитов было 38, на самом деле они соответсуют всего 4 категориям.

### Категоризация данных
<a id='part_0.2'></a>

Теперь нам предстоит категоризировать данные.

Начнем с создания функцию, с помощью которой мы сможем добавить в первоначальную базу данных добавить категории целей кредитов и примения ее к *df*.

In [22]:
def purpose_to_category(row):
    for i in range(purpose_dict.shape[0]):
        if row['purpose'] == purpose_dict.loc[i,'p_unique']:
            return purpose_dict.loc[i,'p_category']

df['category']=df.apply(purpose_to_category, axis=1)
df.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,total_income_id,category
0,1,-8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,1,жилье/недвижимость
1,1,-4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,1,автомобиль
2,0,-5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,1,жилье/недвижимость
3,3,-4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,1,образование
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,1,свадьба


Также нам необходима категория для оценки зависимости наличия у клиента детей и возвратом кредита в срок. 
У нас будут категории:
* yes - дети есть
* no - детей нет
* error - ошибочные данные

Для удобство создадим отдельный столбец *having_children*.

In [23]:
def new_column_having_children(row):
    if row['children'] > 0:
        return('yes')
    elif row['children'] == 0:
        return('no')
    else:
        return('error')

df['having_children'] = df.apply(new_column_having_children, axis=1)
df.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,total_income_id,category,having_children
0,1,-8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,1,жилье/недвижимость,yes
1,1,-4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,1,автомобиль,yes
2,0,-5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,1,жилье/недвижимость,no
3,3,-4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,1,образование,yes
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,1,свадьба,no


Одна из задач проекта состоит в том, чтобы изучить зависимость между уровнем дохода и возвратом кредита в срок. Напоминаю, что тут необходимо смотреть только тех клиентов, у кого первоначально были указаны средние значения дохода (столбец *total_income_id* равен 1).

Посмотрим максимальные, минимальные, среднюю и медианную зарплату.

In [24]:
salary_max = df[df['total_income_id'] == 1]['total_income'].max()
salary_min = df[df['total_income_id'] == 1]['total_income'].min()
salary_mean = df[df['total_income_id'] == 1]['total_income'].mean()
salary_median = df[df['total_income_id'] == 1]['total_income'].median()

print(salary_max)
print(salary_min)
print(salary_mean)
print(salary_median)

2265604
20667
167421.80171567362
145017.0


Так как медианное значение составило 145 тысяч, выберем как средний уровень дохода значения примерно от 145 * 0,5 до 145 * 1,5.

Таким образом получаем четыре категории:
* низкий - менее 70 000 рублей
* средний - от 70 000 до 215 000 рублей
* высокий - Свыше 215 000 рублей
* нет данных - те клиенты, по которым первоначально не были предоставлены данные о доходе

Создадим функцию, распределяющую категории в зависимости от доходов клиента и учитывающую информацию в столбце *total_income_id*, то есть были ли предоставлены данные о среднем доходе.
Применим ее к *df* для получения нового столбца с категориями дохода.

In [25]:
def new_column_total_income_category(row):
    if row['total_income_id'] == 0:
        return('нет данных')
    salary = row['total_income']
    if salary < 70000:
        return('низкий')
    if salary <= 215000:
        return('средний')
    if salary > 215000:
        return('высокий')

df['total_income_category'] = df.apply(new_column_total_income_category, axis=1)
df.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,total_income_id,category,having_children,total_income_category
0,1,-8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,1,жилье/недвижимость,yes,высокий
1,1,-4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,1,автомобиль,yes,средний
2,0,-5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,1,жилье/недвижимость,no,средний
3,3,-4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,1,образование,yes,высокий
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,1,свадьба,no,средний


**Вывод**

Мы создали дополнительные столбцы с категориями для того, чтоб иметь возможность ответить на поставленные задаяи проекта и сделать анализ данных клиентов банка.

# Анализ данных

[*Назад к оглавлению*](#content)

### Есть ли зависимость между наличием детей и возвратом кредита в срок?
<a id='part_1.1'></a>

Сгруппируем таблицу по наличию детей (столбец *having_children*) и посчитаем среднее значение задолженности.

In [26]:
df.groupby('having_children')['debt'].mean()

having_children
error    0.021277
no       0.075438
yes      0.092537
Name: debt, dtype: float64

Мы видим, что у нас появились значения *error*. Это значит, что некоторые данные были предоставлены некорректно. Проверим теперь значения в зависимости от количества детей.

In [27]:
df.groupby('children')['debt'].mean()

children
-1     0.021277
 0     0.075438
 1     0.092346
 2     0.094542
 3     0.081818
 4     0.097561
 5     0.000000
 20    0.105263
Name: debt, dtype: float64

Мы можем наблюдать два нетипичных значения: -1 ребенок и 20 детей. 20 детей - маловероятно, но в принципе возможно. -1 - это явная ошибка в данных, о чем необходимо сообщить коллегам, занимавшия выгрузкой данных.

Посмотрим число таких ошибочных знаяений.

In [28]:
df[df['children'] == -1]['children'].count()

47

Таких строк немного, поэтому существенного влияния на результаты они не окажут.

Также проверим численность клиентов с 5 и 20 детьми.

In [29]:
df[df['children'] == 5]['children'].count()

9

In [30]:
df[df['children'] == 20]['children'].count()

76

9 человек с 5 детьми - вполне реальное число. Обратим внимание, что люди, имеющие  детей крайне ответственно относятся к кредитам - ни один из них не задерживал выплаты.

76 человек с 20 детьми - больше походе на ошибку, хотя с уверенностью сказать невозможно. Необходимо передать информацию коллегам и уточнить, верны ли данные.

**Вывод**

Даже в таком простом для заполнения столбце как количество детей могут появляться ошибки. Необходимо изучить, почему они появляются и как избежать подобного в дальнейшем.

В целом мы можем заметить, что у клиентов имеющих детей больше проблем с возвратом кредитов в срок, чем у бездетных. Возможно, это связано с тем, что при появлении в семье ребенка сущетсвенно возрастают траты и возвращать кредит в срок становится сложнее.

### Есть ли зависимость между семейным положением и возвратом кредита в срок?
<a id='part_1.2'></a>

Сгруппируем таблицу по семейному положению (столбец *family_status*), посчитаем среднее значение задолженности и отсортируем.

In [31]:
df.groupby('family_status')['debt'].mean().sort_values()

family_status
вдовец / вдова           0.065693
в разводе                0.071130
женат / замужем          0.075452
гражданский брак         0.093471
Не женат / не замужем    0.097509
Name: debt, dtype: float64

**Вывод**

Можно заметить, что у людей, потерявших супруга (вдовцы и разведенные) наилучшие показатели выплаты кредитов. Худшие показатели у людей в гражданском браке и незамужних.
Определенный вывод по отношению к наличию у заемщика партнера и показателю выплаты кредитов сделать сложно.

### Есть ли зависимость между уровнем дохода и возвратом кредита в срок?
<a id='part_1.3'></a>

Сгруппируем таблицу по уровню дохода (столбец *total_income_category*), посчитаем среднее значение задолженности и отсортируем.

In [32]:
df.groupby('total_income_category')['debt'].mean().sort_values()

total_income_category
низкий        0.068521
высокий       0.070040
нет данных    0.080837
средний       0.086052
Name: debt, dtype: float64

**Вывод**

Мы можем заметить, что хуже всего показатели задолженности у клиентов со средним доходом. Возможно, потому что они уже повысили уровень жизни, но заработная плата пока не позволяет покрыывать все кредиты.
Лучше всего отдают кредиты клиенты с низким доходом. Они умеют считать свои деньги и ответствено относятся к кредитам.

### Как разные цели кредита влияют на его возврат в срок?
<a id='part_1.4'></a>

Сгруппируем таблицу по целям кредита (столбец *category*), посчитаем среднее значение задолженности и отсортируем.

In [33]:
df.groupby('category')['debt'].mean().sort_values()

category
жилье/недвижимость    0.072334
свадьба               0.080034
образование           0.092200
автомобиль            0.093590
Name: debt, dtype: float64

**Вывод**

Лучше всего дела обстоят с кредитами на жилье и недвижимость. Вероятно, это связано с тем, что люди стараются здраво оценивать свои силы в оплате таких крупных покупок, кредиты по которым обычно необходимо оплачивать много лет.

Хуже всего обстановка с кредитами на авто и образование. Проблемы с выплатами кредитов на автомобили, возможно, связана с тем, что клиенты первоначально берут кредит, который будет заведомо тяжело платить, думая лишь о сиюминутной пользе.
Проблемы с оплатами кредитов на обучение могут быть связаны с тем, что клиенты в итоге после обучения не получают заработную плату, на которую рассчитывали, брав данный кредит.

# Вывод 
<a id='finish'></a>

[*Назад к оглавлению*](#content)

Первое, на что следует обратить внимание - это наличие ошибок в данных.

В ходе анализа были обнаружены следующие проблемы:
* Столбец *days_employed* с информацией о трудовом стаже клиента в днях был выгружен некорректно. Многие значения являются отрицательными, или же стаж составляет около тысячи лет, что невозможно. Необходимо проверить выгрузку и скорректировать ее.
* Были обнаружены пропуски в данных в столбцах *days_employed* и *total_income* со стажами клиентов и среднем доходе. Необходимо выяснить, с чем связаны данные пропуски (ошибка выгрузки или же отстутвие данных в базе).
* Наличие полностью дублирующихся строк. Появление дубликатов могло быть связано как с "человеческим фактором", таким как повторное занесение клиента в базу, так и с проблемами выгрузки. Необходимо выяснить этот вопрос с ответственными людбми.
* Ошибки в значениях в столбце *children*. При анализе были обнаружены отрицательные значения, чего в действительности быть не может. Также было обнаружено удивительно много клиентов с большим количеством детей (20). Надо проверить, в чем было дело и при необходимости исправить информацию в базе данных. 

Также хотелось бы предложить в дальнейшем предоставлять уникальный идентификатор клиента. Это может помочь в более качественном удалении дубликатов и поиске ошибок в данных.

По результатам проведенного анализа данных мы получили следующие выводы: 
* Клиенты, не имеющие детей, более ответственны с выплатами кредитов в срок, чем те, у кого есть дети. Причем количество детей от 1 до 4 особенного влияния не имеет - все клиенты с детьми примерно одинакого задерживают выплаты. Работники с 5 детьми самые дисциплинированные - ни одной задолженности. Однако это может быть связано исключительно с тем, что выборка по таким клиентам крайне мала.
* Одназначного вывода по зависимости возвратов кредитов и наличию у клиента партнера сделать нельзя. В то время как у разведенных в вдовцов самые лучшие показатели возвратов кредитов, у незамужних - значение возвратов наихудшая. Также не скажешь ничего конкретного про людей в официальном и в гражданском браке - люди в официальном браке в середине списка, в то время как значенияу клиентов в гражданском браке существенно хуже. Возможно, клиенты, имеющие или имевшие официальный статус отношений, более ответственно относятся к кредитам.
* Клиенты с низким доходом определенно лучше справляются с возвратом кредитов в срок. Возможно, это связано с тем, что таким клиентам обычно одобряют не очень большие суммы кредитов, которые куда проще выплатить. Хуже всего дела обстоят у улиентов со средним доходом - они задерживают оплату куда чаще остальных.
* Меньше всего задержек выплат у клиентов, взявших кредит на покупку жидья и недвижимости. Скорее всего это связано с наибольшей ответственностью: так как в случае сильной просрочки по кредиту банк может забрать купленную в кредит недвижимость, котоая может оказаться единственным жильем. Хуже всего дела обстоят с кредитами на авто и образование. Проблемы с выплатами кредитов на автомобили, возможно, связана с тем, что клиенты первоначально берут кредит, который будет заведомо тяжело платить, думая лишь о сиюминутной пользе. Проблемы с оплатами кредитов на обучение могут быть связаны с тем, что клиенты в итоге после обучения не получают заработную плату, на которую рассчитывали, брав данный кредит.
